In [2]:
import torch
import pandas as pd
import sklearn

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import pipeline

import sys
sys.path.append('../sentence-sentimental')

from dataset.datasets import SentimentalDataset
from metrics.metrics import compute_metrics

from sklearn.datasets import load_iris # 샘플 데이터 로딩
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
MODEL_NAME = 'klue/roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'clas

In [4]:
data = pd.read_csv('/opt/ml/finance_sentiment_corpus/finance_data.csv')
data['labels'] = data['labels'].map({'negative':0, 'neutral':1, 'positive':2})

train_encoding = tokenizer(
    data['kor_sentence'].tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

train_set = SentimentalDataset(train_encoding, data['labels'])

In [5]:
print(len(data['kor_sentence']), len(data['labels']))

4846 4846


In [6]:
sentence_train, sentence_test, label_train, label_test = train_test_split(data['kor_sentence'], data['labels'],
                                                                            test_size=0.2, 
                                                                            shuffle=True, stratify=data['labels'], # label에 비율을 맞춰서 분리
                                                                            random_state=34)

In [7]:
train_encoding = tokenizer(
    sentence_train.tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

test_encoding = tokenizer(
    sentence_test.tolist(),
    return_tensors='pt',
    padding=True,
    truncation=True
    )

In [9]:
train_set = SentimentalDataset(train_encoding, label_train)
test_set = SentimentalDataset(test_encoding, label_test)

AttributeError: 'SentimentalDataset' object has no attribute 'len'

In [26]:
sentence_train

1828                핀란드 식품업계 기업들은 우크라이나가 제공하는 기회에 관심이 많다.
4809    펜틱+아이넨은 미디어하우스가 제공하는 인터넷 콘텐츠 대부분이 영원히 무료일 수는 없...
1166                    또한 이러한 시장에 대한 6년간의 역사적 분석이 제공됩니다.
277     루키 총리는 핀란드 사본리나에 위치한 키론살미 해협을 가로지르는 철제 구조물의 인도...
4670    2009년 8월 3일 핀란드 미디어 그룹 Ilkka-Yhtyma Oyj(헬: ILK...
                              ...                        
3515    알스트롬 주식거래소 7.2.2.27 10시 30분에 발표 알스트롬 주식회사의 총 5...
4503           라우트 대변인은 "다른 인력에 대한 적응 조치는 당분간 적절하다"고 말했다.
434     주식수 증가는 2006년 스톡옵션제도에 따라 회사 경영에 부여된 주식선택권에 따른 ...
3212         개발사업단도 지난해 11월 대비 개발사업단가를 3분의 1가량 낮추겠다고 밝혔다.
3773                    이 건물에는 제품 개발 및 테스트 실험실이 들어설 예정이다.
Name: kor_sentence, Length: 3876, dtype: object

In [8]:
type(data)

numpy.ndarray

In [9]:
type(train_set)

dataset.datasets.SentimentalDataset